In [ ]:
!pip install requests
!pip install transformers
!pip install nltk

In [ ]:
import nltk
nltk.download('all')

In [ ]:
import requests
from transformers import pipeline
import datetime
import re
import time
from nltk import word_tokenize, pos_tag



In [ ]:
def isInFutureTenseOrPresentParticiple(sentence):
    text = word_tokenize(sentence)
    tagged = pos_tag(text)
    for (word, tag) in tagged:
        if(tag in ["MD", "VB", "VBG"]):
            return True

    return False


isInFutureTenseOrPresentParticiple("Tomorrow I will go to school")

In [ ]:
def get_sentiment(keyword, date, responseDict):
    nude_keyword = re.sub('[^A-Za-z0-9]+', '', keyword).lower()
    classifier = pipeline('sentiment-analysis')


    month = date.month
    year = date.year
    sleepTime = 10
    key = (month, year)

    if( key not in responseDict.keys()):
        while(True):
            url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key=cOZRb3HZAYcI4njj6RGnDirpFWJy9twH"
            response = requests.get(url)

            if(response.status_code != 200):
                time.sleep(sleepTime)       # To not get blocked
                sleepTime = sleepTime + 1
                print(f"Updating sleeptime to : {sleepTime}")
            else:
                responseDict.update({key: response})
                print(f"Saved a resposne on {date}")
                break
    
    all_data = responseDict[key].json()
    articles = all_data["response"]["docs"]

    sum_score = 0
    count = 0
    on_date_article = 0

    for entry in articles:
        headline = entry["headline"]["main"]
        if(isInFutureTenseOrPresentParticiple(headline) == False):
            continue

        pub_datetime = datetime.datetime.strptime(entry["pub_date"], '%Y-%m-%dT%H:%M:%S+%f')
        pub_date = datetime.date(pub_datetime.year, pub_datetime.month, pub_datetime.day)

        if(pub_date == date ):
            nude_headline = re.sub('[^A-Za-z0-9]+', '', headline).lower()
            on_date_article = on_date_article + 1

            if(nude_keyword in nude_headline):
                #print(f"Relevant headline: {headline}")
                
                sentiment_obj = classifier(headline)
                sign = 1 
                if(sentiment_obj[0]["label"] == "NEGATIVE"):
                    sign = -1
                sum_score = sum_score + (sign * sentiment_obj[0]["score"])
                count = count + 1
    
    print(f"#Article published on {date} is {on_date_article}")
    if(count == 0):
        return 0
    else: 
        #print(f"Entry: {sum_score/count}")
        return sum_score / count
    

# Testing Code
keyword = "covid"
date = datetime.date(2021, 1, 27) # 2021-01-27
dummyDict = dict()

scr = get_sentiment(keyword, date, dummyDict)
#date = date + datetime.timedelta(days = 1)
#scr = get_sentiment(keyword, date, dummyDict)

print(scr)

In [ ]:
# Both dates are inclusive
def getSeries(keyword, startDate, endDate, file_name):
    series = []
    responseDict = dict()

    while(startDate <= endDate):
        scr = get_sentiment(keyword, startDate, responseDict)
        series.append(scr)

        file_append = open(file_name, "a")
        file_append.write(f"{startDate}, {scr}\n")
        file_append.close()

        startDate = startDate + datetime.timedelta(days=1)
    
    
    return series 

file_name =  "output.txt"
file_create = open(file_name, "w")
file_create.write("date,score\n")
file_create.close()

# 2020-01-13
# 2021-03-07
startDate = datetime.date(2020, 1, 13)
endDate = datetime.date(2021, 3, 7)
sentiment_series = getSeries("covid", startDate, endDate, file_name)

#print(sentiment_series)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import math

print(sentiment_series)


mini_srs = []
gap = 30
for i in range(0, len(sentiment_series), gap):
    sum = 0
    for j in range(i, i + gap):
        if(j >= len(sentiment_series)):
            break
        sum += sentiment_series[j]
    mini_srs.append(1 - sum/ gap)


s = pd.Series(mini_srs)
p = plt.plot(s)